In [1]:
%reload_ext autoreload
%autoreload 2
# import pandas as pd
import numpy as np

from code_.process_conll import process_file, advanced_process_file, extract_features

from code_.bert import Tokenizer, convert_to_dataset, compute_metrics, get_labels_list_from_dataset, task, batch_size, model_checkpoint
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from datasets import load_metric

c:\Users\snipercapt\Desktop\ANLP\AdvancedNLP\Task_3\code_\bert.py:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")
c:\ProgramData\Anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
df_val = process_file('data/raw/en_ewt-up-dev.conllu')
df_train = process_file('data/raw/small_en-ewt-up-train.conllu')
# df_train = process_file('data/raw/en_ewt-up-train.conllu')
df_test = process_file('data/raw/en_ewt-up-test.conllu')

process_file(): dataframe len: 4979
process_file(): dataframe len: 4110
process_file(): dataframe len: 4802


In [3]:
# df_val = advanced_process_file('data/raw/en_ewt-up-dev.conllu')
# df_train = advanced_process_file('data/raw/en_ewt-up-train.conllu')
# df_test = advanced_process_file('data/raw/en_ewt-up-test.conllu')

In [4]:
df_test

,sentence,predicate,pred columns,labels
0,"[What, if, Google, Morphed, Into, GoogleOS, ?]",Morphed,11,"_, _, ARG1, V, _, ARG2, _"
1,"[What, if, Google, expanded, on, its, search, ...",expanded,11,"_, _, ARG0, V, _, _, _, _, _, _, _, _, _, _, A..."
2,"[(, And, ,, by, the, way, ,, is, anybody, else...",way,11,"_, _, _, _, _, V, _, _, _, _, _, _, _, _, _, _..."
3,"[(, And, ,, by, the, way, ,, is, anybody, else...",is,12,"_, _, _, _, _, ARGM-DIS, _, V, ARG1, _, _, _, ..."
4,"[(, And, ,, by, the, way, ,, is, anybody, else...",was,13,"_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _..."
...,...,...,...,...
4797,"[He, listens, and, is, excellent, in, diagnosi...",explaining,16,"ARG0, _, _, _, _, _, _, _, _, _, V, _, _, ARG1..."
4798,"[He, listens, and, is, excellent, in, diagnosi...",issues,17,"_, _, _, _, _, _, _, _, _, _, _, _, ARGM-ADJ, ..."
4799,"[He, listens, and, is, excellent, in, diagnosi...",suggesting,18,"ARG0, _, _, _, _, _, _, _, _, _, _, _, _, _, _..."
4800,"[He, listens, and, is, excellent, in, diagnosi...",exercises,19,"_, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _..."


In [5]:
dataset = convert_to_dataset(df_train, df_val, df_test)

In [6]:
labels_list = get_labels_list_from_dataset(dataset)
print(sorted(labels_list, key=lambda x: len(x)))

['V', '_', 'C-V', 'ARGA', 'ARG3', 'ARG2', 'ARG5', 'ARG0', 'ARG4', 'ARG1', 'C-ARG4', 'C-ARG2', 'R-ARG2', 'C-ARG0', 'R-ARG0', 'C-ARG1', 'C-ARG3', 'R-ARG3', 'R-ARG1', 'ARGM-EXT', 'ARGM-NEG', 'ARGM-DIS', 'ARGM-PRD', 'ARGM-PRP', 'ARGM-ADV', 'ARGM-GOL', 'ARGM-REC', 'ARG1-DSP', 'ARGM-MNR', 'ARGM-PRR', 'ARGM-DIR', 'ARGM-LVB', 'ARGM-TMP', 'ARGM-ADJ', 'ARGM-MOD', 'ARGM-COM', 'ARGM-LOC', 'ARGM-CAU', 'ARGM-CXN', 'C-ARG1-DSP', 'R-ARGM-DIR', 'R-ARGM-LOC', 'R-ARGM-ADV', 'R-ARGM-MNR', 'C-ARGM-EXT', 'C-ARGM-LOC', 'C-ARGM-CXN', 'R-ARGM-ADJ', 'R-ARGM-TMP', 'R-ARGM-CAU', 'C-ARGM-MNR', 'R-ARGM-COM']


In [7]:
tok = Tokenizer(model_checkpoint, labels_list)

In [8]:
tok.tokenize_and_align_labels_pred(dataset['test'])

IndexError: list index out of range

In [ ]:
tokenized_datasets = dataset.map(tok.tokenize_and_align_labels_pred, batched=True)

Map:   0%|          | 0/4110 [00:00<?, ? examples/s]

Map:   0%|          | 0/4979 [00:00<?, ? examples/s]

Map:   0%|          | 0/4802 [00:00<?, ? examples/s]

In [9]:
# initialise model
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(labels_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# or load it from checkpoint
# model = AutoModelForTokenClassification.from_pretrained('model_checkpoints/no_pred/checkpoint-3500')

In [23]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"model_checkpoints/small_no_pred",
    # evaluation_strategy = 'epoch',
    evaluation_strategy = 'steps',
    eval_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,
    weight_decay=0.01,
    push_to_hub=False,
)

In [24]:
data_collator = DataCollatorForTokenClassification(tok.tokenizer)
metric = load_metric("seqeval")

In [25]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tok.tokenizer,
    compute_metrics=compute_metrics
)

In [26]:
# for i in range(len(tokenized_datasets['train'])):
#     if len(tokenized_datasets['train'][i]['labels']) != len(tokenized_datasets['train'][i]['input_ids']):
#         print(i)
#         print(len(tokenized_datasets['train'][i]['labels']))
#         print(len(tokenized_datasets['train'][i]['input_ids']))
#         print()


In [27]:
print(tokenized_datasets['train'][0]['input_ids'])

[101, 2632, 1011, 23564, 2386, 1024, 2137, 2749, 2730, 21146, 28209, 14093, 2632, 1011, 2019, 2072, 1010, 1996, 14512, 2012, 1996, 8806, 1999, 1996, 2237, 1997, 1053, 4886, 2213, 1010, 2379, 1996, 9042, 3675, 1012, 102]


In [28]:
trainer.train()

  0%|          | 0/258 [00:00<?, ?it/s]

  0%|          | 0/156 [00:00<?, ?it/s]

C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: _ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: V seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG0 seems not to be NE tag.
  warnings.

{'eval_loss': 1.2949837446212769, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.8603919004937465, 'eval_runtime': 172.1958, 'eval_samples_per_second': 28.915, 'eval_steps_per_second': 0.906, 'epoch': 0.08}


KeyboardInterrupt: 

In [ ]:
# results, plots, reports etc.

In [40]:
trainer.evaluate()

  0%|          | 0/312 [00:00<?, ?it/s]

C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: _ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: V seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\snipercapt\AppData\Roaming\Python\Python311\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: ARG0 seems not to be NE tag.
  warnings.

{'eval_loss': 0.7728551030158997, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.8654313626565152, 'eval_runtime': 162.8888, 'eval_samples_per_second': 30.567, 'eval_steps_per_second': 1.915, 'epoch': 0.03}


{'eval_loss': 0.7728551030158997,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_f1': 0.0,
 'eval_accuracy': 0.8654313626565152,
 'eval_runtime': 162.8888,
 'eval_samples_per_second': 30.567,
 'eval_steps_per_second': 1.915,
 'epoch': 0.03}

In [29]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [labels_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [labels_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

  0%|          | 0/156 [00:00<?, ?it/s]

{'ADJ': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 243},
 'ADV': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 477},
 'ARG0': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 63},
 'ARG1': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 120},
 'ARG2': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 11},
 'ARG3': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8},
 'ARGM-ADV': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ARGM-CAU': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ARGM-COM': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'ARGM-CXN': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7},
 'ARGM-EXT': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2},
 'ARGM-LOC': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 13},
 'ARGM-MNR': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 3},
 'ARGM-TMP': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 8},
 'CAU': {

In [30]:
predictions

array([[4, 4, 4, ..., 0, 0, 0],
       [4, 4, 4, ..., 0, 0, 0],
       [4, 4, 4, ..., 0, 0, 0],
       ...,
       [4, 4, 4, ..., 0, 0, 0],
       [4, 4, 4, ..., 0, 0, 0],
       [4, 4, 4, ..., 0, 0, 0]], dtype=int64)

'ARGM-LOC'